# Chapter13. RDD 고급 개념

- 집계와 키-값 형태의 RDD
- 사용자 정의 파티셔닝
- RDD 조인

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.2:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1686093283879)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3aaf14d6


In [2]:
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

myCollection: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)


In [3]:
val words = spark.sparkContext.parallelize(myCollection, 2)

words: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at <console>:24


# 13.1 키-값 형태의 기초(키-값 형태의 RDD)
- RDD에는 데이터를 키-값 형태로 다룰 수 있는 다양한 메서드가 있음
    - 이러한 메서드는 <연산명>ByKey 형태의 이름을 가짐
    - 메서드 이름에 ByKey가 있다면 PairRDD 타입만 사용할 수 있음
    - PairRDD 타입을 만드는 가장 쉬운 방법은 RDD에 맵 연산을 수행해 키-값 구조로 만드는 것
    - 즉, 레코드에 두 개의 값이 존재

In [4]:
words.map(word => (word.toLowerCase, 1))

res1: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1] at map at <console>:26


## 13.1.1 keyBy

In [5]:
val keyword = words.keyBy(word => word.toLowerCase.toSeq(0).toString)

keyword: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[2] at keyBy at <console>:24


## 13.1.2 값 매핑하기
- 만약 튜플 형태의 데이터를 사용한다면 스파크는 튜플의 첫 번째 요소를 키로, 두 번째 요소를 값으로 추정

In [6]:
keyword.mapValues(word => word.toUpperCase).collect()

res2: Array[(String, String)] = Array((s,SPARK), (t,THE), (d,DEFINITIVE), (g,GUIDE), (:,:), (b,BIG), (d,DATA), (p,PROCESSING), (m,MADE), (s,SIMPLE))


In [7]:
//flatmap 함수를 사용해 반환되는 결과의 각 로우가 문자를 나타내도록 확장할 수 있음
keyword.flatMapValues(word => word.toUpperCase).collect()

res3: Array[(String, Char)] = Array((s,S), (s,P), (s,A), (s,R), (s,K), (t,T), (t,H), (t,E), (d,D), (d,E), (d,F), (d,I), (d,N), (d,I), (d,T), (d,I), (d,V), (d,E), (g,G), (g,U), (g,I), (g,D), (g,E), (:,:), (b,B), (b,I), (b,G), (d,D), (d,A), (d,T), (d,A), (p,P), (p,R), (p,O), (p,C), (p,E), (p,S), (p,S), (p,I), (p,N), (p,G), (m,M), (m,A), (m,D), (m,E), (s,S), (s,I), (s,M), (s,P), (s,L), (s,E))


## 13.1.3 키와 값 추출하기

In [8]:
keyword.keys.collect()

res4: Array[String] = Array(s, t, d, g, :, b, d, p, m, s)


In [9]:
keyword.values.collect()

res5: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)


## 13.1.4 lookup
- 특정 키에 관한 결과를 찾기
    - 그러나 각 입력에 대해 오직 하나의 키만 찾을 수 있도록 강제하는 메커니즘은 없음

In [10]:
keyword.lookup("s")

res6: Seq[String] = WrappedArray(Spark, Simple)


## 13.1.5 sampleByKey
- 근사치나 정확도를 이용해 키를 기반으로 RDD 샘플을 생성할 수 있음
- 특정 키를 부분 샘플링할 수 있으며 선택에 따라 비복원 추출을 사용할 수도 있음

In [11]:
val distinctChars = words.flatMap(word => word.toLowerCase.toSeq).distinct.collect()

distinctChars: Array[Char] = Array(d, p, t, b, h, n, f, v, :, r, l, s, e, a, i, k, u, o, g, m, c)


In [12]:
import scala.util.Random

val sampleMap = distinctChars.map(c => (c, new Random().nextDouble())).toMap

import scala.util.Random
sampleMap: scala.collection.immutable.Map[Char,Double] = Map(e -> 0.4375463899951435, s -> 0.6673301150653165, n -> 0.673849374110227, t -> 0.5510348197633197, u -> 0.6174184873558151, f -> 0.7642923286632328, a -> 0.658489020265458, m -> 0.2896779594638964, i -> 0.6882052046882807, v -> 0.3032258433656986, b -> 0.5462155668570206, g -> 0.83621646234217, l -> 0.22473109345403064, p -> 0.4024886231352476, c -> 0.4982075442532208, h -> 0.2973042221059691, r -> 0.126795826003682, : -> 0.5812962361524623, k -> 0.9418188180232476, o -> 0.7479822120139356, d -> 0.22010257052046878)


In [13]:
words.map(word => (word.toLowerCase.toSeq(0), word)).sampleByKey(true, sampleMap, 6L).collect()

res7: Array[(Char, String)] = Array((s,Spark), (t,The), (d,Definitive), (g,Guide), (:,:))


In [14]:
words.map(word => (word.toLowerCase.toSeq(0), word)).sampleByKeyExact(true, sampleMap, 6L).collect()

res8: Array[(Char, String)] = Array((s,Spark), (t,The), (d,Definitive), (g,Guide), (:,:), (b,Big), (p,Processing), (m,Made), (s,Simple))


# 13.2 집계

In [15]:
val chars = words.flatMap(word => word.toLowerCase.toSeq)
val KVcharacters = chars.map(letter => (letter,1))

def maxFunc(left:Int, right:Int) = math.max(left, right)
def addFunc(left:Int, right:Int) = left+right

val nums = sc.parallelize(1 to 30, 5)

chars: org.apache.spark.rdd.RDD[Char] = MapPartitionsRDD[20] at flatMap at <console>:26
KVcharacters: org.apache.spark.rdd.RDD[(Char, Int)] = MapPartitionsRDD[21] at map at <console>:27
maxFunc: (left: Int, right: Int)Int
addFunc: (left: Int, right: Int)Int
nums: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[22] at parallelize at <console>:32


## 13.2.1 countByKey
- 각 키의 아이템 수를 구하고 로컬 맵으로 결과를 수집

In [16]:
val timeout = 1000L //밀리세컨드 단위
val confidence = 0.95

KVcharacters.countByKey()
KVcharacters.countByKeyApprox(timeout, confidence)

timeout: Long = 1000
confidence: Double = 0.95
res9: org.apache.spark.partial.PartialResult[scala.collection.Map[Char,org.apache.spark.partial.BoundedDouble]] = (final: Map(e -> [7.000, 7.000], s -> [4.000, 4.000], n -> [2.000, 2.000], t -> [3.000, 3.000], u -> [1.000, 1.000], f -> [1.000, 1.000], a -> [4.000, 4.000], m -> [2.000, 2.000], i -> [7.000, 7.000], v -> [1.000, 1.000], b -> [1.000, 1.000], g -> [3.000, 3.000], l -> [1.000, 1.000], p -> [3.000, 3.000], c -> [1.000, 1.000], h -> [1.000, 1.000], r -> [2.000, 2.000], : -> [1.000, 1.000], k -> [1.000, 1.000], o -> [1.000, 1.000], d -> [4.000, 4.000]))


## 13.2.2 집계 연산 구현 방식 이해하기
- 구현 방식은 잡의 안정성을 위해 매우 중요
- groupBy와 reduce 함수의 비교를 통해 이해해보기

## groupByKey
- 각 키의 총 레코드 수를 구하는 경우 groupByKey의 결과로 만들어진 그룹에 map 연산을 수행하는 방식이 가장 좋아 보임

In [17]:
KVcharacters.groupByKey().map(row => (row._1, row._2.reduce(addFunc))).collect()

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3252/0x0000000801c94e40.arg$1/putField, from class java.lang.Object (module java.base)

- 모든 익스큐터에서 함수를 적용하기 전에 해당 키와 관련된 모든 값을 메모리로 읽어 들여야 함
- 심각하게 치우쳐진 키가 있다면 일부 파티션이 엄청난 양의 값을 가질 수 있으므로 OutOfMemoryError가 발생

## reduceByKey

In [18]:
KVcharacters.reduceByKey(addFunc).collect()

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3282/0x0000000801ca5828.arg$1/putField, from class java.lang.Object (module java.base)

- flatMap을 동일하게 수행한 다음 map 연산을 사용해 각 문자의 인스턴스를 1로 매핑
- 그런 다음 결괏값을 배열에 모을 수 있도록 합계 함수와 함께 reduceByKey 메서드를 수행
- 이러한 구현 방식은 각 파티션에서 리듀스 작업을 수행하기 때문에 훨씬 안정적이며 모든 값을 메모리에 유지하지 않아도 됨
- 또한 최종 리듀스 과정을 제외한 모든 작업은 개별 워커에서 처리하기 때문에 연산 중 셔플이 발생하지 않음
- 안정성과 연산 수행 속도가 크게 향상됨

## 13.2.3 기타 집계 메서드

### aggregate

In [19]:
nums.aggregate(0)(maxFunc, addFunc)

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3325/0x0000000801cbd448.arg$1/putField, from class java.lang.Object (module java.base)

In [20]:
val depth = 3
nums.treeAggregate(0)(maxFunc, addFunc, depth)

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3328/0x0000000801d00420.arg$1/putField, from class java.lang.Object (module java.base)

### aggregateByKey
- 키를 기준으로 연산을 수행

In [21]:
KVcharacters.aggregateByKey(0)(addFunc, maxFunc).collect()

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3495/0x0000000801d4c000.arg$1/putField, from class java.lang.Object (module java.base)

### combineByKey
- 집계 함수 대신 컴바이너를 사용

In [22]:
val valToCombiner = (value:Int) => List(value)
val mergeValuesFunc = (vals:List[Int], valToAppend:Int) => valToAppend :: vals
val mergeCombinerFunc = (vals1:List[Int], vals2:List[Int]) => vals1 ::: vals2

val outputPartitions = 6

valToCombiner: Int => List[Int] = $Lambda$3504/0x0000000801c56000@65c99158
mergeValuesFunc: (List[Int], Int) => List[Int] = $Lambda$3505/0x0000000801d50000@570e26ad
mergeCombinerFunc: (List[Int], List[Int]) => List[Int] = $Lambda$3506/0x0000000801d505a0@1c32a3f5
outputPartitions: Int = 6


In [23]:
KVcharacters.combineByKey(valToCombiner,mergeValuesFunc,mergeCombinerFunc,outputPartitions).collect()

res15: Array[(Char, List[Int])] = Array((f,List(1)), (r,List(1, 1)), (l,List(1)), (s,List(1, 1, 1, 1)), (a,List(1, 1, 1, 1)), (g,List(1, 1, 1)), (m,List(1, 1)), (t,List(1, 1, 1)), (b,List(1)), (h,List(1)), (n,List(1, 1)), (i,List(1, 1, 1, 1, 1, 1, 1)), (u,List(1)), (o,List(1)), (c,List(1)), (d,List(1, 1, 1, 1)), (p,List(1, 1, 1)), (v,List(1)), (:,List(1)), (e,List(1, 1, 1, 1, 1, 1, 1)), (k,List(1)))


### foldByKey
- 결합 함수와 항등원인 '제로값'을 이용해 각 키의 값을 병합

In [24]:
KVcharacters.foldByKey(0)(addFunc).collect()

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3529/0x0000000801d539b8.arg$1/putField, from class java.lang.Object (module java.base)

# 13.3 cogroup
- 스칼라를 사용하는 경우 최대 3개, 파이썬을 사용하는 경우 최대 2개의 키-값 형태의 RDD를 그룹화할 수 있으며 각 키를 기준으로 값을 결합
- 즉, RDD에 대한 그룹 기반의 조인 수행

In [25]:
import scala.util.Random

val distinctChars = words.flatMap(word => word.toLowerCase.toSeq).distinct
val charRDD = distinctChars.map(c => (c, new Random().nextDouble()))
val charRDD2 = distinctChars.map(c => (c, new Random().nextDouble()))
val charRDD3 = distinctChars.map(c => (c, new Random().nextDouble()))

charRDD.cogroup(charRDD2, charRDD3).take(5)

import scala.util.Random
distinctChars: org.apache.spark.rdd.RDD[Char] = MapPartitionsRDD[31] at distinct at <console>:28
charRDD: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[32] at map at <console>:29
charRDD2: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[33] at map at <console>:30
charRDD3: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[34] at map at <console>:31
res17: Array[(Char, (Iterable[Double], Iterable[Double], Iterable[Double]))] = Array((d,(CompactBuffer(0.3762916423043762),CompactBuffer(0.9651337597778552),CompactBuffer(0.6600202097004217))), (p,(CompactBuffer(0.48745951298651935),CompactBuffer(0.4990312446157963),CompactBuffer(0.7153875797973754))), (t,(CompactBuffer(0.737476270447212),CompactBuffer(0.833937437349693),CompactBuff...


# 13.4 조인

## 13.4.1 내부 조인

In [26]:
val keyedChars = distinctChars.map(c => (c, new Random().nextDouble()))
val outputPartitions = 10

KVcharacters.join(keyedChars).count()
KVcharacters.join(keyedChars, outputPartitions).count()

keyedChars: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[37] at map at <console>:29
outputPartitions: Int = 10
res18: Long = 51


## 13.4.2 zip

In [27]:
val numRange = sc.parallelize(0 to 9, 2)
words.zip(numRange).collect()

numRange: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[44] at parallelize at <console>:28
res19: Array[(String, Int)] = Array((Spark,0), (The,1), (Definitive,2), (Guide,3), (:,4), (Big,5), (Data,6), (Processing,7), (Made,8), (Simple,9))


# 13.5 파티션 제어하기
- RDD를 사용하면 데이터가 클러스터 전체에 물리적으로 정확히 분산되는 방식을 정의할 수 있음

## 13.5.1 coalesce
- 파티션을 재분배할 때 발생하는 데이터 셔플을 방지하기 위해 동일한 워커에 존재하는 파티션을 합치는 메서드

In [28]:
words.coalesce(1).getNumPartitions

res20: Int = 1


## 13.5.2 repartition
- 처리 시 노드 간의 셔플이 발생할 수 있음

In [29]:
words.repartition(10)

res21: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[50] at repartition at <console>:29


## 13.5.3 repartitionAndSortWithinPartitions

## 13.5.4 사용자 정의 파티셔닝
- RDD를 사용하는 가장 큰 이유 중 하나
- 잡이 성공적으로 동작되는지 여부에 상당한 영향을 미침
- 대표적인 예 : 페이지랭크
    - 사용자 정의 파티셔닝을 이용해 클러스터의 배치 구조를 제어하고 셔플을 회피
- 데이터 치우침 같은 문제를 피하고자 클러스터 전체에 걸쳐 데이터를 균등하게 분배
- 구조적 API로 RDD를 얻고 사용자 정의 파티셔너를 적용한 다음 다시 DataFrame이나 Dataset으로 변환해야 함

In [30]:
val df = spark.read.option("header", "true").option("inferSchema", "true").csv("./sample_data/retail-data/all/")
val rdd = df.coalesce(10).rdd

df: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]
rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[66] at rdd at <console>:27


In [31]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [32]:
import org.apache.spark.HashPartitioner

rdd.map(r => r(6)).take(5).foreach(println)
val keyedRDD = rdd.keyBy(row => row(6).asInstanceOf[Int].toDouble)

keyedRDD.partitionBy(new HashPartitioner(10)).take(10)

17850
17850
17850
17850
17850


import org.apache.spark.HashPartitioner
keyedRDD: org.apache.spark.rdd.RDD[(Double, org.apache.spark.sql.Row)] = MapPartitionsRDD[68] at keyBy at <console>:30
res23: Array[(Double, org.apache.spark.sql.Row)] = Array((15100.0,[536374,21258,VICTORIAN SEWING BOX LARGE,32,12/1/2010 9:09,10.95,15100,United Kingdom]), (16250.0,[536388,21754,HOME BUILDING BLOCK WORD,3,12/1/2010 9:59,5.95,16250,United Kingdom]), (16250.0,[536388,21755,LOVE BUILDING BLOCK WORD,3,12/1/2010 9:59,5.95,16250,United Kingdom]), (16250.0,[536388,21523,DOORMAT FANCY FONT HOME SWEET HOME,2,12/1/2010 9:59,7.95,16250,United Kingdom]), (16250.0,[536388,21363,HOME SMALL WOOD LETTERS,3,12/1/2010 9:59,4.95,16250,United Kingdom]), (16250.0,[536388,21411,GINGHAM HEART  DOORSTOP RED,3,12/1/2010 9:59,4.25,16250,United Kingdom]), (...


In [33]:
import org.apache.spark.Partitioner

class DomainPartitioner extends Partitioner {
    def numPartitions = 3
    def getPartition(key: Any): Int = {
        val customerId = key.asInstanceOf[Double].toInt
        
        if(customerId == 17850.0 || customerId == 12583.0) {
            return 0
        } else {
            return new java.util.Random().nextInt(2) + 1
        }
    }
}

keyedRDD.partitionBy(new DomainPartitioner).map(_._1).glom().map(_.toSet.toSeq.length).take(5)

import org.apache.spark.Partitioner
defined class DomainPartitioner
res24: Array[Int] = Array(2, 4299, 4308)


# 13.6 사용자 정의 직렬화
- Kyro 직렬화

In [35]:
class SomeClass extends Serializable {
    var someValue = 0
    def setSomeValue(i:Int) = {
        someValue = i
        this
    }
}

sc.parallelize(1 to 10).map(num => new SomeClass().setSomeValue(num))

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$4907/0x0000000802233448.arg$1/putField, from class java.lang.Object (module java.base)